In [1]:
%pwd

'/home/jingw222/myDirPy/MLPractices/Instacart'

In [2]:
%matplotlib inline
import zipfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
pd.options.display.max_rows=200
pd.options.display.max_columns=200

In [4]:
dtype = {'order_id': np.int32, 'product_id': np.uint16, 'add_to_cart_order': np.uint8, 'reordered': np.int8,
        'user_id': np.int32, 'eval_set': np.int8, 'order_number': np.int8, 'order_dow': np.int8, 
         'order_hour_of_day': np.int8, 'days_since_prior_order': np.float32}

In [5]:
ord_pri_train = pd.read_csv('./data/stats/ord_pri_train.csv', dtype=dtype)
ord_trn = pd.read_csv('./data/stats/ord_trn.csv', dtype=dtype)
products = pd.read_csv('./data/stats/products.csv', index_col='product_id')
orders_train_eval1 = pd.read_csv('./data/stats/orders_train_eval1.csv')

In [6]:
# user_size = len(ord_pri_train['user_id'].unique())
# user_idx = np.random.choice(ord_pri_train['user_id'].unique(), size=round(0.4*user_size), replace=False)

# ord_pri_train = ord_pri_train.loc[ord_pri_train['user_id'].isin(user_idx)]

# orders_train_eval1 = orders_train_eval1.loc[orders_train_eval1['user_id'].isin(user_idx)]

In [7]:
ord_pri_train = ord_pri_train.drop(['order_id', 'reordered', 'cumsum_days_since_prior_order'], axis=1).join(products['aisle_id'], on='product_id')
orders_train_eval1 = orders_train_eval1.drop('days_since_prior_order', axis=1)

ord_pri_train = ord_pri_train.merge(orders_train_eval1.drop(['order_id', 'order_number'], axis=1), on='user_id', suffixes=['_trn', '_pred'])

In [8]:
def dow_range(trn, pred):
    return abs(trn-pred) if abs(trn-pred)<=3 else 7-abs(trn-pred)

def hour_range(trn, pred):
    return abs(trn-pred) if abs(trn-pred)<=12 else 24-abs(trn-pred)

dow_range_vec = np.vectorize(dow_range)
hour_range_vec = np.vectorize(hour_range)

ord_pri_train['dow_range'] = dow_range_vec(ord_pri_train['order_dow_trn'], ord_pri_train['order_dow_pred'])
ord_pri_train['hour_range'] = hour_range_vec(ord_pri_train['order_hour_of_day_trn'], ord_pri_train['order_hour_of_day_pred'])

ord_pri_train.drop(['order_dow_trn', 'order_hour_of_day_trn', 'order_dow_pred', 'order_hour_of_day_pred'], 
                   axis=1, inplace=True)

In [9]:
def change_dtypes(df, col, dtype):
    df[col] = df[col].astype(dtype)

In [10]:
change_dtypes(ord_pri_train, ['aisle_id'], np.uint8)
change_dtypes(ord_pri_train, ['dow_range', 'hour_range'], np.int8)

In [11]:
# prod_counts = grouped_['product_id'].value_counts().rename('prod_counts')
# dow_range_out = grouped['dow_range'].unique().apply(lambda x: any(a in x for a in [2, 3])).rename('dow_range_out')
# hour_range_out = grouped['hour_range'].unique().apply(lambda x: any(a in x for a in range(7, 13))).rename('hour_range_out')

# range_out = dow_range_out.to_frame().join(hour_range_out)
# range_out['range_out'] = np.where(((range_out['dow_range_out']==False) & (range_out['hour_range_out']==False)), False, True)

# temp = prod_counts.to_frame().join(range_out['range_out'])
# temp.loc[(temp['prod_counts']!=1) | (temp['range_out']==False), 'keep'] = 1
# temp = temp.drop(['prod_counts', 'range_out'], axis=1)

# ord_pri_train = ord_pri_train.join(temp, on=['user_id', 'product_id']).dropna(subset=['keep']).drop('keep', axis=1)
# ord_pri_train = ord_pri_train.rename(columns={'order_number_trn': 'order_number',
#                                'order_dow_trn': 'order_dow',
#                                'order_hour_of_day_trn': 'order_hour_of_day'})

In [12]:
grouped = ord_pri_train.groupby(['user_id', 'product_id'])
grouped__ = ord_pri_train.groupby(['user_id', 'aisle_id'])
grouped_ = ord_pri_train.groupby(['user_id'])

In [13]:
def consecutive(data, stepsize=1):
    l = np.split(data, np.where(np.diff(data) != stepsize)[0]+1)
    l_len = [len(e) for e in l]
    return np.median(l_len)

In [14]:
tot_prod_count_by_user = grouped_['product_id'].count().rename('tot_prod_count_by_user')

In [15]:
cart_order_by_user = grouped['add_to_cart_order'].mean().rename('cart_order_by_user').astype(np.float32).round(3)
cart_order_by_user = cart_order_by_user.apply(lambda x: 1/np.sqrt(x))

In [24]:
last_order_num_user = grouped_['order_number'].max().rename('last_order_num_user')
last_order_num_prod = grouped['order_number'].max().rename('last_order_num_prod')

last_order_num_by_prod_user = (pd.merge(last_order_num_prod.reset_index(), last_order_num_user.reset_index(), on='user_id')
                               .set_index(['user_id', 'product_id']))

In [17]:
prod_count_by_user = grouped['product_id'].count().rename('prod_count_by_user')
prod_pct_by_user = (prod_count_by_user / tot_prod_count_by_user).rename('prod_pct_by_user').astype(np.float16).round(3)

prod_ratio_by_user = (prod_count_by_user / last_order_num_user).rename('prod_ratio_by_user').astype(np.float16).round(3)

prod_metrics = pd.concat((prod_pct_by_user, prod_ratio_by_user), axis=1)

del prod_count_by_user, prod_pct_by_user, prod_ratio_by_user

In [18]:
# isin_order_fn = lambda g: [int(last_order_num_user[g.name[0]] in set(g['order_number'])),
#                           int(last_order_num_user[g.name[0]]-1 in set(g['order_number'])),
#                           int(last_order_num_user[g.name[0]]-2 in set(g['order_number'])),
#                           int(last_order_num_user[g.name[0]]-3 in set(g['order_number'])),
#                           int(1 in set(g['order_number'])),
#                           int(2 in set(g['order_number'])),
#                           int(3 in set(g['order_number'])),
#                           int(4 in set(g['order_number']))]

# isin_orders = grouped.apply(isin_order_fn).rename('isin_orders')

# isin_orders_ = pd.DataFrame([x for x in isin_orders.to_frame().isin_orders], index=isin_orders.index)
# isin_orders_.columns = ['isin_last_order','isin_sed_last_order','isin_thd_last_order','isin_fth_last_order',
#                         'isin_first_order','isin_sed_order','isin_thd_order','isin_fth_order']

# isin_orders_.to_csv('./data/stats/isin_orders.csv')

In [19]:
dtypes = {'isin_last_order': np.int16,'isin_sed_last_order': np.int16,'isin_thd_last_order': np.int16,
          'isin_fth_last_order': np.int16, 'isin_first_order': np.int16,'isin_sed_order': np.int16,
          'isin_thd_order': np.int16,'isin_fth_order': np.int16}

isin_orders = pd.read_csv('./data/stats/isin_orders.csv', dtype=dtypes, index_col=['user_id', 'product_id'])

/home/jingw222/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [20]:
# range_by_prod = grouped.apply(lambda g: [np.mean(g['hour_range']), np.mean(g['dow_range'])]).rename('range_by_prod')

# range_by_prod_ = pd.DataFrame([x for x in range_by_prod.to_frame().range_by_prod], index=range_by_prod.index)
# range_by_prod_.columns = ['range_hour_by_prod', 'range_dow_by_prod']

# range_by_prod_.to_csv('./data/stats/range_by_prod.csv')

In [21]:
range_by_prod = pd.read_csv('./data/stats/range_by_prod.csv', index_col=['user_id', 'product_id'])

/home/jingw222/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [26]:
last_order_num_by_prod_user['last_order_num_prod_diff1'] = np.subtract(last_order_num_by_prod_user['last_order_num_user'], last_order_num_by_prod_user['last_order_num_prod'])
last_order_num_by_prod_user['last_order_num_prod_diff2'] = np.divide(last_order_num_by_prod_user['last_order_num_prod'], last_order_num_by_prod_user['last_order_num_user'])

last_order_num_by_prod_user.drop(['last_order_num_prod', 'last_order_num_user'], axis=1, inplace=True)

last_order_num_by_prod_user['last_order_num_prod_diff1'] = 1/np.sqrt(last_order_num_by_prod_user['last_order_num_prod_diff1']+1) 

In [33]:
orders_train_eval1 = orders_train_eval1.set_index('user_id').drop('order_number', axis=1)
orders_train_eval1.rename(columns = {'order_dow': 'pred_order_dow', 'order_hour_of_day': 'pred_order_hour_of_day'}, inplace=True)

In [36]:
merged = pd.concat((cart_order_by_user, 
                    isin_orders,
                    range_by_prod,
                    last_order_num_by_prod_user), axis=1)

In [39]:
merged = merged.join(orders_train_eval1.drop('order_id', axis=1)).join(products[['aisle_id']]).join(prod_metrics)

In [44]:
merged

cart_order_by_user  isin_last_order  isin_sed_last_order  \
user_id product_id                                                             
1       196                   0.845154                1                    1   
        10258                 0.547750                1                    1   
        10326                 0.447214                0                    0   
        12427                 0.550482                1                    1   
        13032                 0.397370                1                    0   
        13176                 0.408248                0                    0   
        14084                 0.707107                0                    0   
        17122                 0.408248                0                    0   
        25133                 0.500000                1                    1   
        26088                 0.471405                0                    0   
        26405                 0.447214                0                    0   
        30450                 0.447214                0                    0   
        35951                 0.377964                1                    0   
        38928                 0.500000                1                    0   
        39657                 0.577350                1                    0   
        41787                 0.377964                0                    0   
        46149                 0.577350                1                    1   
        49235                 0.534522                0                    1   
2       23                    0.288675                0                    0   
        79                    0.577350                0                    1   
        1559                  0.365148                1                    0   
        2002                  0.312348                0                    0   
        2573                  0.308607                1                    0   
        3151                  0.377964                0                    0   
        4071                  0.707107                0                    0   
        4957                  0.242536                0                    0   
        5212                  0.258199                0                    0   
        5322                  0.707107                0                    0   
        5450                  0.242536                0                    0   
        5869                  0.500000                0                    1   
        5907                  0.377964                0                    0   
        7781                  0.377964                1                    0   
        7963                  0.250000                1                    0   
        8138                  0.408248                0                    0   
        8296                  0.288675                0                    0   
        8479                  0.577350                0                    0   
        9124                  0.208514                0                    0   
        9681                  0.365148                0                    0   
        10305                 0.316228                0                    0   
        12000                 0.620174                0                    0   
        12258                 0.229416                0                    0   
        13176                 0.288675                0                    0   
        13351                 0.316228                0                    0   
        13742                 0.377964                0                    0   
        14306                 0.250000                0                    0   
        14553                 0.353553                0                    0   
        15841                 0.333333                0                    0   
        16521                 0.408248                0                    1   
        16589       

In [45]:
orders_train_eval1

,order_id,pred_order_dow,pred_order_hour_of_day
user_id,,,
1,1187899,4,8
2,1492625,1,11
5,2196797,0,11
7,525192,2,11
8,880375,1,14
9,1094988,6,10
10,1822501,0,19
13,1827621,0,21
14,2316178,2,19


In [46]:
order_id_user_map = orders_train_eval1.drop(['pred_order_dow', 'pred_order_hour_of_day'], axis=1).reset_index().set_index('order_id')

In [49]:
order_id_user_map.sort_index()

,user_id
order_id,
1,112108
36,79431
38,42756
96,17227
98,56463
112,125030
170,182389
218,98711
226,51011


In [50]:
ord_trn = ord_trn.loc[ord_trn['reordered']==1]

In [52]:
ord_trn['order_id'].nunique()

122607

In [53]:
order_id_user_map['user_id'].nunique()

131209

In [55]:
ord_trn = ord_trn.join(order_id_user_map, on='order_id')

In [56]:
ord_trn = ord_trn.set_index(['user_id', 'product_id']).drop('order_id', axis=1).sort_index()


reordered
user_id product_id           
1       196                 1
        10258               1
        13032               1
        25133               1
        26088               1
        26405               1
        38928               1
        39657               1
        46149               1
        49235               1
2       5450                1
        7963                1
        16589               1
        22559               1
        22825               1
        22963               1
        24852               1
        32792               1
        33957               1
        41787               1
        45066               1
        45613               1
5       15349               1
        21413               1
        21616               1
        40706               1
7       13198               1
        17638               1
        29894               1
        37999               1
        40852               1
        43967               1
        45066               1
        47272               1
8       15937               1
        21903               1
        23165               1
        41540               1
9       1559                1
        8467                1
        8834                1
        10132               1
        12075               1
        17600               1
        18926               1
        20899               1
        24954               1
        26604               1
        27555               1
        27596               1
        27973               1
        29594               1
        30252               1
        30967               1
        33754               1
        38988               1
        40571               1
        41844               1
        42347               1
        42828               1
13      4210                1
        19934               1
        27086               1
        27435               1
14      3808                1
        8744                1
        15172               1
        15869               1
        28601               1
        29509               1
        32115               1
        37434               1
17      12720               1
        16797               1
        18534               1
18      21137               1
        22031               1
        25997               1
        36216               1
        47546               1
21      10957               1
        12683               1
        25740               1
        32645               1
        44632               1
23      13544               1
        22021               1
        33819               1
        42372               1
        48205               1
        49306               1
24      31222               1
27      1323                1
        2966                1
        4920                1
        12745               1
        21137               1
        21386               1
        32263               1
        33754               1
...                       ...
206181  33316               1
        41950               1
206185  14852               1
        40153               1
206186  2091                1
        25466               1
        41290               1
206188  6473                1
        12078               1
        34487               1
        35898               1
        36186               1
        44560               1
206190  8518                1
        20549               1
        21616               1
        23484               1
        26283               1
        40706               1
        46979               1
        47209               1
206191  10880               1
        21267               1
        21567               1
        21903               1
        22935               1
        24009               1
        24838               1
        24852               1
        24964               1
        27104               1


In [66]:
ord_trn.head(20)

reordered
user_id product_id           
1       196                 1
        10258               1
        13032               1
        25133               1
        26088               1
        26405               1
        38928               1
        39657               1
        46149               1
        49235               1
2       5450                1
        7963                1
        16589               1
        22559               1
        22825               1
        22963               1
        24852               1
        32792               1
        33957               1
        41787               1

In [59]:
merged = merged.join(ord_trn)

In [61]:
merged.fillna(0, inplace=True)

In [63]:
merged['reordered'] = merged['reordered'].astype(np.int16)

In [65]:
merged

cart_order_by_user  isin_last_order  isin_sed_last_order  \
user_id product_id                                                             
1       196                   0.845154                1                    1   
        10258                 0.547750                1                    1   
        10326                 0.447214                0                    0   
        12427                 0.550482                1                    1   
        13032                 0.397370                1                    0   
        13176                 0.408248                0                    0   
        14084                 0.707107                0                    0   
        17122                 0.408248                0                    0   
        25133                 0.500000                1                    1   
        26088                 0.471405                0                    0   
        26405                 0.447214                0                    0   
        30450                 0.447214                0                    0   
        35951                 0.377964                1                    0   
        38928                 0.500000                1                    0   
        39657                 0.577350                1                    0   
        41787                 0.377964                0                    0   
        46149                 0.577350                1                    1   
        49235                 0.534522                0                    1   
2       23                    0.288675                0                    0   
        79                    0.577350                0                    1   
        1559                  0.365148                1                    0   
        2002                  0.312348                0                    0   
        2573                  0.308607                1                    0   
        3151                  0.377964                0                    0   
        4071                  0.707107                0                    0   
        4957                  0.242536                0                    0   
        5212                  0.258199                0                    0   
        5322                  0.707107                0                    0   
        5450                  0.242536                0                    0   
        5869                  0.500000                0                    1   
        5907                  0.377964                0                    0   
        7781                  0.377964                1                    0   
        7963                  0.250000                1                    0   
        8138                  0.408248                0                    0   
        8296                  0.288675                0                    0   
        8479                  0.577350                0                    0   
        9124                  0.208514                0                    0   
        9681                  0.365148                0                    0   
        10305                 0.316228                0                    0   
        12000                 0.620174                0                    0   
        12258                 0.229416                0                    0   
        13176                 0.288675                0                    0   
        13351                 0.316228                0                    0   
        13742                 0.377964                0                    0   
        14306                 0.250000                0                    0   
        14553                 0.353553                0                    0   
        15841                 0.333333                0                    0   
        16521                 0.408248                0                    1   
        16589       

In [67]:
merged.to_csv('./data/train_merged_with_index.csv')

In [68]:
merged.to_csv('./data/train_merged_without_index.csv', index=False)

## Feature Engineering (Added)